<a href="https://colab.research.google.com/github/dcolinmorgan/grph/blob/main/bench_cody.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# gpu demo for cody

In [ ]:
!pip install --extra-index-url=https://pypi.nvidia.com cuml-cu11 cudf-cu11 cugraph-cu11 pylibraft_cu11 raft_dask_cu11 dask_cudf_cu11 pylibcugraph_cu11 pylibraft_cu11
import cuml,cudf
print(cuml.__version__)

!wget https://www.dropbox.com/s/owzp6af8v4vcp1s/sample-state-with-severity.csv.zip
!unzip sample-state-with-severity.csv.zip

!pip install -U --force git+https://github.com/graphistry/pygraphistry.git@cudf
!pip install -U git+https://github.com/graphistry/cu-cat.git@DT
!pip install dirty_cat 
!pip install umap-learn

!nvidia-smi --query-gpu=gpu_name --format=csv
!pip install nvsmi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.nvidia.com
     ━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/1.1 GB 195.9 MB/s eta 0:00:05

In [ ]:
import cuml,cudf
print(cuml.__version__)


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
import os
from collections import Counter
import cProfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pstats import Stats
import cudf
from time import time
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth', 200)

In [ ]:
import graphistry
graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username='dcolinmorgan', password='fXjJnkE3Gik6BWy') ## key id, secret key

# graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username='dcolinmorgan', password='***') ## key id, secret key
graphistry.__version__

import cu_cat
print(cu_cat.__file__)

In [ ]:
# !nvidia-smi --query-gpu=gpu_name --format=csv
!nvidia-smi --query-gpu=gpu_name --format=csv,noheader
# !nvidia-smi -L
# import sys
# import os

# gpu=(sys.argv[1])
# gpu
import os

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:

filtered_state_df=pd.read_csv("sample-state-with-severity.csv").drop(columns=["Unnamed: 0"])

# filtered_state_df.to_parquet('drugs/'+s+'_drugs.pq',compression='gzip',index=False)
for d in pd.unique(filtered_state_df.DRUG_NAME)[0:2]:
    filtered_state_df["DRUG_NAME"] = filtered_state_df["DRUG_NAME"].str.replace('\t', '', regex=True)
    # d='TAPENTADOL'
    filtered_df = filtered_state_df[(filtered_state_df['DRUG_NAME']=='FENTANYL')]



    filtered_df=filtered_df.rename(columns={"('CALC_BASE_WT_IN_GM', 'sum')":"calc_weight_per_month_grams","('DOSAGE_UNIT', 'sum')":"pills_per_month" })
    filtered_df["month"]=pd.to_datetime(filtered_df["month_date"],format='%Y-%m').dt.month
    filtered_df["year"]=pd.to_datetime(filtered_df["month_date"],format='%Y-%m').dt.year
    filtered_df.month_date = filtered_df.month_date.values.astype('datetime64[M]')
    filtered_df=filtered_df.set_index("month_date")
    filtered_df = filtered_df.sort_index()

    filtered_df["trailing six month avg"]=filtered_df.groupby(["BUYER_DEA_NO","DRUG_NAME","dos_str","month_date"])['pills_per_month'].transform("sum").rolling("183d",min_periods=1).mean()
    filtered_df["yearly_average"]=filtered_df.groupby(["BUYER_DEA_NO","DRUG_NAME","dos_str","year"])['pills_per_month'].transform('mean')
    filtered_df['2*TRAILING12']=2*filtered_df.groupby(["BUYER_DEA_NO","DRUG_NAME","dos_str","month_date"])['pills_per_month'].transform("sum").rolling(window="365D",min_periods=1).mean()
    filtered_df['3*TRAILING12']=3*filtered_df.groupby(["BUYER_DEA_NO","DRUG_NAME","dos_str","month_date"])['pills_per_month'].transform("sum").rolling(window="365D",min_periods=1).mean()
    filtered_df["Max monthly dosage units"]=8000
    filtered_df=filtered_df.reset_index()
    filtered_df["exceeds_max_monthly_dosage_unit_threshhold"] = filtered_df['pills_per_month'] > filtered_df["Max monthly dosage units"]
    filtered_df["exceeds_trailing_6_month_avg"] = filtered_df['pills_per_month'] > filtered_df["trailing six month avg"]
    filtered_df["exceeds_3*TRAILING12_threshold"] = filtered_df['pills_per_month'] > filtered_df['3*TRAILING12']
    filtered_df["exceeds_2*TRAILING12_threshold"] = filtered_df['pills_per_month'] > filtered_df['2*TRAILING12']
    filtered_df["exceeds_yearly_avg"] = filtered_df['pills_per_month'] > filtered_df['yearly_average']


    sev_lst = dict()
    for index in filtered_df.index:
        sev_lst[index] = (filtered_df["exceeds_max_monthly_dosage_unit_threshhold"][index].sum() +
                          filtered_df["exceeds_trailing_6_month_avg"][index].sum() +
                          (2*filtered_df["exceeds_3*TRAILING12_threshold"][index].sum()) +
                          filtered_df["exceeds_2*TRAILING12_threshold"][index].sum() +
                          filtered_df["exceeds_yearly_avg"][index].sum())/5
    filtered_df["severity_level"] = sev_lst



    rows=filtered_df.groupby(['month','year','BUYER_DEA_NO','dos_str']).sum().reset_index()


In [ ]:
rows[['BUYER_STATE','DRUG_NAME','BUYER_BUS_ACT','BUYER_COUNTY','BUYER_DEA_NO','ds']]=filtered_df[filtered_df.columns[filtered_df.dtypes.eq('object')].tolist()]
rows['month_date']=filtered_df.month_date
rows.ds=pd.to_datetime(rows.ds)
s = rows.select_dtypes(include='int').columns
rows[s] = rows[s].astype("float")

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:


s = rows.select_dtypes(include='int').columns
rows[s] = rows[s].astype("float")
rows.dtypes

In [ ]:

rows=rows.drop(columns=['month_date','ds'])
g = graphistry.nodes(cudf.from_pandas(rows))
X = ['DRUG_NAME',"severity_level",'BUYER_BUS_ACT',"BUYER_COUNTY"]

# rows['feats'] = rows[X].astype(str).agg(' '.join, axis=1)
g2=g.featurize(feature_engine='cu_cat',memoize=True)
g3=g2.umap(X=X,engine='cuml')

g3.bind(node=filtered_df.month_date)

In [ ]:
# g3._nodes['month_date']=filtered_df.month_date
g3.plot()

In [ ]:
# filtered_state_df=pd.read_parquet('/content/drive/MyDrive/graphistry/drugs/CA_drugs.pq')

# A=pd.DataFrame(columns=['state','drug','gpu','dc-cuml'])
# files=glob.glob('/content/drive/MyDrive/graphistry/drugs/*_drugs.pq')
# # states = ["AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
# # for s in states:
#     # filtered_state_df=pd.read_parquet('/content/drive/MyDrive/graphistry/drugs/'+s+'_drugs.pq')
# for f in files:
#     filtered_state_df=pd.read_parquet(f)
#     s=os.path.basename(f).str.strip('.pq')
#     # state_df = ddf[ddf["BUYER_STATE"].isin([s])]
#     # filtered_state_df=state_df.compute().to_pandas()
#     # filtered_state_df.to_parquet(s+'_drugs.pq',compression='gzip',index=False)
#     for d in pd.unique(filtered_state_df.DRUG_NAME):
#         filtered_state_df["DRUG_NAME"] = filtered_state_df["DRUG_NAME"].str.replace('\t', '', regex=True)
#         # d='TAPENTADOL'
#         filtered_df = filtered_state_df[(filtered_state_df['DRUG_NAME']==d)]



#         filtered_df=filtered_df.rename(columns={"('CALC_BASE_WT_IN_GM', 'sum')":"calc_weight_per_month_grams","('DOSAGE_UNIT', 'sum')":"pills_per_month" })
#         filtered_df["month"]=pd.to_datetime(filtered_df["month_date"],format='%Y-%m').dt.month
#         filtered_df["year"]=pd.to_datetime(filtered_df["month_date"],format='%Y-%m').dt.year
#         filtered_df.month_date = filtered_df.month_date.values.astype('datetime64[M]')
#         filtered_df=filtered_df.set_index("month_date")
#         filtered_df = filtered_df.sort_index()

#         filtered_df["trailing six month avg"]=filtered_df.groupby(["BUYER_DEA_NO","DRUG_NAME","dos_str","month_date"])['pills_per_month'].transform("sum").rolling("183d",min_periods=1).mean()
#         filtered_df["yearly_average"]=filtered_df.groupby(["BUYER_DEA_NO","DRUG_NAME","dos_str","year"])['pills_per_month'].transform('mean')
#         filtered_df['2*TRAILING12']=2*filtered_df.groupby(["BUYER_DEA_NO","DRUG_NAME","dos_str","month_date"])['pills_per_month'].transform("sum").rolling(window="365D",min_periods=1).mean()
#         filtered_df['3*TRAILING12']=3*filtered_df.groupby(["BUYER_DEA_NO","DRUG_NAME","dos_str","month_date"])['pills_per_month'].transform("sum").rolling(window="365D",min_periods=1).mean()
#         filtered_df["Max monthly dosage units"]=8000
#         filtered_df=filtered_df.reset_index()
#         filtered_df["exceeds_max_monthly_dosage_unit_threshhold"] = filtered_df['pills_per_month'] > filtered_df["Max monthly dosage units"]
#         filtered_df["exceeds_trailing_6_month_avg"] = filtered_df['pills_per_month'] > filtered_df["trailing six month avg"]
#         filtered_df["exceeds_3*TRAILING12_threshold"] = filtered_df['pills_per_month'] > filtered_df['3*TRAILING12']
#         filtered_df["exceeds_2*TRAILING12_threshold"] = filtered_df['pills_per_month'] > filtered_df['2*TRAILING12']
#         filtered_df["exceeds_yearly_avg"] = filtered_df['pills_per_month'] > filtered_df['yearly_average']


#         sev_lst = dict()
#         for index in filtered_df.index:
#             sev_lst[index] = (filtered_df["exceeds_max_monthly_dosage_unit_threshhold"][index].sum() +
#                               filtered_df["exceeds_trailing_6_month_avg"][index].sum() +
#                               (2*filtered_df["exceeds_3*TRAILING12_threshold"][index].sum()) +
#                               filtered_df["exceeds_2*TRAILING12_threshold"][index].sum() +
#                               filtered_df["exceeds_yearly_avg"][index].sum())/5
#         filtered_df["severity_level"] = sev_lst



#         rows=filtered_df.groupby(['month','year','BUYER_DEA_NO','dos_str']).sum().reset_index()
#         if rows.shape[0]<500000:
#             g = graphistry.nodes(cudf.from_pandas(rows))
#             t=time()
#             g2=g.featurize(feature_engine='cu_cat',memoize=True)
#             t1=time()-t
#             t=time()
#             g2.umap(engine='cuml')
#             t2=time()-t

#             # pd.DataFrame([['gpu',d,t1,t2,samp.shape[0]]]).to_csv('bench3.txt',mode='a',sep='\t',index=None,header=None)

#             g = graphistry.nodes((rows))
#             t=time()
#             g3=g.featurize(feature_engine='dirty_cat',memoize=True)
#             t3=time()-t
#             t=time()
#             # try:
#             g3.umap( engine='umap_learn')
#             # except:
#                 # pass
#             del g, g3,g2
#             t4=time()-t
#             pd.DataFrame([[s,d,t1,t2,t3,t4,rows.shape[0]]]).to_csv('T4_eks_bench.txt',mode='a',sep='\t',index=None,header=None)


In [ ]:
# filtered_df=pd.read_csv("sample-state-with-severity.csv").drop(columns=["Unnamed: 0"])
# samp=filtered_df.sample(500000)

# from graphistry.features import ModelDict, topic_model

# X = ['DRUG_NAME',"severity_level",'BUYER_BUS_ACT',"BUYER_COUNTY","BUYER_STATE"] #buyer_state and county only for multi states

# # samp['feats'] = samp[X].astype(str).agg(' '.join, axis=1)
# samp['month_date']=pd.to_datetime(samp['month_date'])
# samp['day'] = samp['month_date'].dt.day
# samp['month'] = samp['month_date'].dt.month
# samp['year'] = samp['month_date'].dt.year

# sampA=samp.drop(columns=['month_date','ds'])

# # for j,d in enumerate(pd.unique(sampA.DRUG_NAME)):
# for j,d in enumerate(['OXYCODONE','HYDROCODONE']):
#     samp=sampA[sampA['DRUG_NAME']==d]
#     if samp.shape[0]: #<80000:
#         g = graphistry.nodes(cudf.from_pandas(samp))
#         t=time()
#         g2=g.featurize(feature_engine='cu_cat',memoize=True)
#         t1=time()-t
#         t=time()
#         g2.umap(engine='cuml')
#         t2=time()-t
        
#         # pd.DataFrame([['gpu',d,t1,t2,samp.shape[0]]]).to_csv('bench3.txt',mode='a',sep='\t',index=None,header=None)
        
#         g = graphistry.nodes((samp))
#         t=time()
#         g3=g.featurize(feature_engine='dirty_cat',memoize=True)
#         t3=time()-t
#         t=time()
#         # try:
#         g3.umap( engine='umap_learn')
#         # except:
#             # pass
#         del g, g3,g2
#         t4=time()-t
#         pd.DataFrame([[d,t1,t2,t3,t4,samp.shape[0]]]).to_csv('bench80k.txt',mode='a',sep='\t',index=None,header=None)
#         # pd.DataFrame([['cpu',d,t3,t4,samp.shape[0]]]).to_csv('bench3.txt',mode='a',sep='\t',index=None,header=None)

In [ ]:
data=pd.read_csv('bench80k.txt',sep='\t',header=None)
data.columns=['drug','cu_cat','cuml','dirty_cat','umap_learn','dims']

In [ ]:
# data['umap_learn']=data.cpu-data.dirty_cat
data['gpu']=data.cu_cat+data.cuml
data['cpu']=data.dirty_cat+data.umap_learn

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# !cp *.txt /content/drive/MyDrive/graphistry/

In [ ]:
# data=pd.read_csv('bench5.txt',sep='\t',names=['drug','cu_cat','cuml','dirty_cat','cpu','dims'])

RR=data.sort_values(by='cpu',ascending=False)#.to_csv('drug_bench.txt',sep='\t',index=False)
RR['drug_count']= RR.drug+'_'+RR.dims.astype(str) #.str.split(',').str[0].str.strip('(')
# RR['umap_learn']=RR.cpu-RR.dirty_cat


In [ ]:
RR#.dims.astype(str)

In [ ]:
A=np.sum(RR.gpu)
B=np.sum(RR.cpu)
print('total gpu:' +str(A))
print('total cpu:' +str(B))
print('ratio:' +str(B/A))

In [ ]:
# RR.drop(columns=['drug','dims','cpu','gpu'],inplace=True)


import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(12, 4))

RR.index=RR['drug_count']
RR[['drug_count','cu_cat','cuml']].plot.bar(ax=axes[0],stacked=True)
RR[['drug_count','dirty_cat','umap_learn']].plot.bar(ax=axes[1],stacked=True)

axes[0].set_ylim([0, 20])
axes[0].set_title('GPU time')
axes[0].set_ylabel('seconds')
axes[0].set_xlabel('drug and row count')
# axes[0].set_yscale('log')

axes[1].set_ylim([0, 500])
# axes[1].set_yscale('log')
axes[1].set_title('CPU time')
axes[1].set_xlabel('drug and row count')
fig.suptitle('Massachusetts')
fig.autofmt_xdate(rotation=45)